In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import make_column_selector as selector
import joblib
from flow_process import Preprocessor
from datasets import load_dataset
# 1) Load
norm_data = Preprocessor("CIC-IDS2017")
df_all = load_dataset("c01dsnap/CIC-IDS2017")
df_norm_colums = norm_data.norm_name_columns(df_all["train"])

c:\Users\dangn\miniconda3\envs\ddos\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
unique_labels = df_norm_colums.unique("Label")
print(unique_labels)
print("Số lượng class:", len(unique_labels))

['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration', 'Web Attack � Brute Force', 'Web Attack � XSS', 'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed']
Số lượng class: 15


In [3]:
for lbl in unique_labels:
    print(repr(lbl))

'BENIGN'
'DDoS'
'PortScan'
'Bot'
'Infiltration'
'Web Attack � Brute Force'
'Web Attack � XSS'
'Web Attack � Sql Injection'
'FTP-Patator'
'SSH-Patator'
'DoS slowloris'
'DoS Slowhttptest'
'DoS Hulk'
'DoS GoldenEye'
'Heartbleed'


In [5]:
df_map_labels = norm_data.mapping_label(df_norm_colums)
print(df_map_labels.columns)
importance_colums = [
    "PSH Flag Count", "min_seg_size_forward", "Flow IAT Max", "Flow IAT Min", "ACK Flag Count", "Destination Port", "Bwd Packet Length Mean", "Bwd Packet Length Max", "Bwd Packet Length Min", "Init_Win_bytes_forward",
    "Fwd IAT Max", "Idle Mean", "Idle Max", "Avg Bwd Segment Size", "Bwd Packet Length Std", "Bwd IAT Mean", "Fwd IAT Std", "Down/Up Ratio", "Max Packet Length", "Average Packet Size",
    "Min Packet Length", "Packet Length Std", "Fwd Packets/s", "Packet Length Mean", "Flow IAT Std", "URG Flag Count", "FIN Flag Count", "Fwd Packet Length Min", "Subflow Fwd Packets", "Bwd IAT Max",
    "Packet Length Variance", "Fwd IAT Mean", "Flow Duration", "Fwd IAT Total", "Bwd IAT Std", "Flow IAT Mean"
]
importance_colums = importance_colums + ["Label"] + ["label_mapped"]
select_columns = norm_data.select_columns(df_map_labels,importance_colums)
output_path = "selected_dataset.csv"
# Xuất ra file CSV
select_columns.to_csv(output_path, index=False, encoding="utf-8")
# select_columns.columns

['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed']
Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Le